In [73]:
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8')

In [13]:
with open('../datasets/Shakespeare/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

## Tokeniser

In [21]:
def get_pair_counts(tokens):

    pair_counts = {}
    for pair in zip(tokens, tokens[1:]):
        pair_counts[pair] = pair_counts.get(pair, 0) + 1

    return pair_counts

def replace_pair(arr, pair, replacement):

    new_arr = []
    i = 0
    while i < len(arr):
        if i < len(arr) - 1 and arr[i] == pair[0] and arr[i+1] == pair[1]:
            new_arr.append(replacement)
            i += 2
        else:
            new_arr.append(arr[i])
            i += 1

    return new_arr

In [54]:
class BPETokeniser:

    def __init__(self):
        self.stoi = {}
        self.vocab = {}
        self.merges = {}

    def decode(self, ids):
        """ given ids (list of integers), return Python string """
        tokens = "".join(self.vocab[idx] for idx in ids)
        return tokens

    def encode(self, string):
        toks = [self.stoi[x] for x in string]
        while len(toks) >= 2: # avoid error when only 1 token is passed
            pair_counts = get_pair_counts(toks)
            pair = min(pair_counts, key=lambda p: self.merges.get(p, float("inf")))
            if pair not in self.merges:
                break # no more possible merges

            new_token = self.merges[pair]
            toks = replace_pair(toks, pair, new_token)
        return toks

    def train(self, text, desired_vocab_size):
        unique_chars = sorted(list(set(text)))
        self.current_vocab_size = len(unique_chars)

        self.stoi = { ch:i for i,ch in enumerate(unique_chars) }
        self.vocab = { i:ch for i,ch in enumerate(unique_chars) }

        tokens = [self.stoi[x] for x in list(text)]

        num_merges = desired_vocab_size - self.current_vocab_size
        ids = list(tokens)

        for i in tqdm(range(num_merges), desc="Merges: ", leave=False):
            pair_counts = get_pair_counts(ids)
            max_occuring_pair = max(pair_counts, key=pair_counts.get)
            
            if pair_counts[max_occuring_pair] == 1:
                print("Max pair only occurs once, breaking from loop")
                break

            new_token_code = self.current_vocab_size + i
            ids = replace_pair(ids, max_occuring_pair, new_token_code)
            self.merges[max_occuring_pair] = new_token_code
        
        for (p0, p1), idx in self.merges.items(): # ordered dictionaries baby (ly python >= 3.8)
            self.vocab[idx] = self.vocab[p0] + self.vocab[p1]


        print(f"Completed, with compression ratio: {len(tokens) / len(ids)}x")

In [55]:
bpe = BPETokeniser()
bpe.train(text, desired_vocab_size=128)

Merges:   0%|          | 0/63 [00:00<?, ?it/s]

Completed, with compression ratio: 1.4957015446547222x


## Transformer

In [32]:
data = torch.tensor(bpe.encode(text), dtype=torch.long)

In [33]:
n = int(0.9 * len(data))
train_data = data[:n]
test_data = data[n:]

In [34]:
def get_batch(split="train"):
    data = train_data if split == 'train' else test_data

    ix = torch.randint(len(data) - block_size, (batch_size, )) # function call is torch.randint(low=0, high, size, ...)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    x, y = x.to(device), y.to(device)

    return x, y

In [35]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

## Transformer

In [91]:
# Hyperparams
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 2500
eval_interval = 100
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
embd_dims = 128
n_heads = 8
n_layers = 8
dropout = 0.3
vocab_size = 128

torch.manual_seed(1337)
# ------------

In [79]:
class SelfAttentionHead(nn.Module):

    def __init__(self, head_size):
        super(SelfAttentionHead, self).__init__()

        self.head_size = head_size

        self.queries = nn.Linear(embd_dims, head_size)
        self.keys = nn.Linear(embd_dims, head_size)
        self.values = nn.Linear(embd_dims, head_size)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)


    def forward(self, x):
        B, T, C = x.shape

        k = self.keys(x)
        q = self.queries(x)

        # print(k.shape, q.shape)

        affs = q @ k.transpose(-2, -1) * C**-0.5 # Divide by C^(-0.5) so that affs has unit var
        affs = affs.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # Present should not be able to communicate with future
        affs = F.softmax(affs, dim=-1) # Normalize
        affs = self.dropout(affs)

        v = self.values(x)
        return affs @ v

In [80]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super(MultiHeadAttention, self).__init__()
        self.sa_heads = nn.ModuleList([SelfAttentionHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(embd_dims, embd_dims)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([head(x) for head in self.sa_heads], dim=-1) # Concat along channel dim
        out = self.dropout(self.proj(out))
        return out

In [82]:
class SingleFeedForward(nn.Module):

    def __init__(self, in_size, out_size):

        super(SingleFeedForward, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(in_size, out_size),
            nn.ReLU(),
            nn.Linear(out_size, in_size), # Project back so that it can be added with the residual pathway
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)


In [83]:
class TransformerBlock(nn.Module):

    def __init__(self, embd_dims, n_heads):
        super(TransformerBlock, self).__init__()

        head_size = embd_dims // n_heads
        self.sa = MultiHeadAttention(n_heads, head_size)
        self.ffwd = SingleFeedForward(embd_dims, 4 * embd_dims)

        self.ln1 = nn.LayerNorm(embd_dims)
        self.ln2 = nn.LayerNorm(embd_dims)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # Skip connection with x added
        x = x + self.ffwd(self.ln2(x)) # Another skip connection
        return x

In [84]:
class DecoderTransformer(nn.Module):

    def __init__(self):

        super(DecoderTransformer, self).__init__()

        self.token_embedding = nn.Embedding(vocab_size, embd_dims)
        self.position_embedding = nn.Embedding(block_size, embd_dims)
        self.blocks = nn.Sequential(*[TransformerBlock(embd_dims, n_heads) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(embd_dims) # layer norm after all the transformer stuff
        # self.lm_head = nn.Linear(embd_dims, vocab_size) # output head of language model
        self.lm_head = nn.Sequential(
            nn.Linear(embd_dims, 50),
            nn.Linear(50, 50),
            nn.Linear(50, vocab_size)
        )

    def forward(self, x, targets=None):
        B, T = x.shape

        tok_emb = self.token_embedding(x)
        pos_emb = self.position_embedding(torch.arange(T, device=device))

        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)

            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, x, max_new_tokens):

        self.eval()
        for _ in range(max_new_tokens):
            cropped_x = x[:, -block_size:] # Only take block size amount of context
            logits, loss = self(cropped_x)
            logits = logits[:, -1, :] # Only get last token for each batch
            probs = F.softmax(logits, dim=-1)
            next_choice = torch.multinomial(probs, num_samples=1) # Predict the next charatcer given the probabilites
            x = torch.concat((x, next_choice), dim=1)
        self.train()
        return x


In [92]:
model = DecoderTransformer().to(device)
print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

epoch_losses = []

for iter in tqdm(range(max_iters), desc="Epoch: "):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    epoch_losses.append(loss.item())

plt.plot(range(max_iters), epoch_losses)

1.62244 M parameters


Epoch:   0%|          | 0/2500 [00:00<?, ?it/s]

step 0: train loss 4.8537, val loss 4.8593
step 100: train loss 4.0669, val loss 4.0517
step 200: train loss 3.8879, val loss 3.8614


KeyboardInterrupt: 

In [69]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(bpe.decode(model.generate(context, max_new_tokens=2000)[0].tolist()))


By hefore in the bukew of thou thour milk, and Ray'd fal
sould lege.

SICGILIA:
There eat fass, that you, det ubver of your pein
done! What to no lord.
Come, Prookaceward my testal Secan oO I athe!

KING EDHE:
Bus wrought had he most will it not of hinould,
Ay more up it Canant in I may oftwards his yescover:
For no here heavanks plast, I am I grest wound
And canrold Praceens content do'd wit; the may unter?

VILLANG:
you dear a that force adgay, hour honour fill
no majourd upt it hald. Plord, eat and lack my are,
Who repans, and da clins!

DUKE 'TIO:
I hath that gint to Harrown.

CLADIUS:
Sit Lave I am goits?

LIXENBEE:
Will fight the plander, I may requifect inch with a hanry dems,
I shall po most asommend, to be full commnest hast my hape.
And any lowat He by the Mackenter.

PLANGE:
Cerlow:
Pige, a Thou one the gegingment entleen.
I'll noble not for beward,ing thour to thesey will.

LUCENTIO:
I city the pitch is atchanten: shall and wrong.
Cake beith ay upon to cannot your ades:
An